In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/pos-data-simulated-restaurant-data/simulated_pos_data_with_seasonal_trends.csv


# QA & Product Support Project for POS Systems
This project demonstrates manual testing, bug tracking, SQL queries, and API testing using a simulated POS dataset from Kaggle.  
Skills: Manual Testing, Regression, Bug Documentation, SQL with Pandas, API Simulation with Requests/Postman-like.  
Dataset: POS Data: Simulated Restaurant Data (from Kaggle).  
Related to Yalla Plus QA role in F&B tech.

In [2]:
import matplotlib.pyplot as plt 
import requests  
import json  
print("Libraries imported successfully!")

Libraries imported successfully!


## Step 2: Load Dataset
Loading the simulated POS data (transactions.csv or similar). This mimics loading data from a POS system for testing.

In [4]:
df = pd.read_csv('/kaggle/input/pos-data-simulated-restaurant-data/simulated_pos_data_with_seasonal_trends.csv')

print("Dataset loaded. First 5 rows:")
display(df.head())

print("\nDataset Info:")
df.info()

Dataset loaded. First 5 rows:


,Timestamp,Customer_ID,Customer_Name,Item_Category,Item_Name,Quantity,Price_Per_Item,Total_Price,Payment_Method
0,2023-06-21 10:35:45,518529,Cassidy David,Cool Drinks,son,4,17.86,71.44,Cash
1,2023-04-23 19:13:55,929362,Kayla Ball,Fruits,commercial,5,7.46,37.30,Credit Card
2,2023-08-13 17:13:43,390753,James Vaughn,Tea,bed,1,8.81,8.81,Debit Card
3,2024-01-24 18:38:37,310934,Eric Spears,Curries,fine,1,12.62,12.62,Credit Card
4,2023-04-09 14:24:55,781110,Hayley King,Ice Cream,her,5,17.69,88.45,Cash



Dataset Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Timestamp       1000 non-null   object 
 1   Customer_ID     1000 non-null   int64  
 2   Customer_Name   1000 non-null   object 
 3   Item_Category   1000 non-null   object 
 4   Item_Name       1000 non-null   object 
 5   Quantity        1000 non-null   int64  
 6   Price_Per_Item  1000 non-null   float64
 7   Total_Price     1000 non-null   float64
 8   Payment_Method  1000 non-null   object 
dtypes: float64(2), int64(2), object(5)
memory usage: 70.4+ KB


In [6]:
negative_prices = df[df['Price_Per_Item'] < 0]
if not negative_prices.empty:
    print("BUG FOUND: Negative prices per item!")
    display(negative_prices)
else:
    print("Test Passed: No negative prices per item.")

total_sales = df['Total_Price'].sum()
print(f"Test Passed: Total Sales = {total_sales}")

duplicates = df[df.duplicated(subset=['Timestamp', 'Customer_ID', 'Item_Name'])]  
if not duplicates.empty:
    print("BUG FOUND: Duplicate transactions!")
    display(duplicates)
else:
    print("Test Passed: No duplicates.")

Test Passed: No negative prices per item.
Test Passed: Total Sales = 37274.479999999996
Test Passed: No duplicates.


In [7]:
df['Timestamp'] = pd.to_datetime(df['Timestamp'])

filtered_df = df[df['Timestamp'] > '2024-01-01']
total_after_date = filtered_df['Total_Price'].sum()
print(f"Query Result: Total Sales after 2024-01-01 = {total_after_date}")

inventory_summary = df.groupby('Item_Name')['Quantity'].sum().reset_index()
print("\nInventory Summary:")
display(inventory_summary)

zero_qty = inventory_summary[inventory_summary['Quantity'] == 0]
if not zero_qty.empty:
    print("BUG FOUND: Items with zero quantity!")
    display(zero_qty)
else:
    print("Test Passed: All items have quantity > 0.")

Query Result: Total Sales after 2024-01-01 = 10098.970000000001

Inventory Summary:


,Item_Name,Quantity
0,American,1
1,Congress,2
2,Democrat,8
3,I,3
4,Mr,3
...,...,...
625,yes,2
626,yet,13
627,young,12
628,your,6


Test Passed: All items have quantity > 0.


In [8]:
bug_report = """
Bug Title: Negative Price Per Item (Simulated for Demo)
Severity: High
Steps to Reproduce:
1. Load dataset.
2. Filter for Price_Per_Item < 0.
Expected: No negative prices.
Actual: Found negative value (simulated example: -5.0 in row X).
Fix Suggestion: Validate input data in POS system.
"""
print(bug_report)

with open('bug_report.md', 'w') as f:
    f.write(bug_report)
print("Bug report saved as bug_report.md")


Bug Title: Negative Price Per Item (Simulated for Demo)
Severity: High
Steps to Reproduce:
1. Load dataset.
2. Filter for Price_Per_Item < 0.
Expected: No negative prices.
Actual: Found negative value (simulated example: -5.0 in row X).
Fix Suggestion: Validate input data in POS system.

Bug report saved as bug_report.md


In [11]:
import plotly.express as px

df['Date'] = df['Timestamp'].dt.date
daily_sales = df.groupby('Date')['Total_Price'].sum().reset_index()

fig = px.line(daily_sales, x='Date', y='Total_Price', title='Daily Sales Trend',
              labels={'Total_Price': 'Total Sales (EGP)', 'Date': 'Date'})
fig.update_traces(line=dict(color='rgb(25,99,132)', width=2.5))
fig.update_layout(
    plot_bgcolor='rgba(0,0,0,0)',
    paper_bgcolor='rgba(0,0,0,0)',
    font=dict(size=14),
    title_font_size=18,
    xaxis_gridcolor='rgba(0,0,0,0.1)',
    yaxis_gridcolor='rgba(0,0,0,0.1)'
)
fig.show()